In [1]:
from enum import Enum, unique
import toml
#from scipy.optimize import fsolve
import numpy as np
from rich.tree import Tree
import pandas as pd

In [2]:
@unique
class ElementTypes(Enum):
    """Element types"""
    SOURCE = 1
    LOAD = 2
    LOSS = 3
    CONVERTER = 4
    LINREG = 5

In [3]:
ElementTypes.SOURCE.name, ElementTypes.SOURCE.value

('SOURCE', 1)

In [144]:
MAX_DEFAULT = 1.0e6
IQ_DEFAULT = 0.0
RS_DEFAULT = 0.0
VDROP_DEFAULT = 0.0
LIMITS_DEFAULT = {'vi': [0, MAX_DEFAULT], 'vo': [0, MAX_DEFAULT], 'ii':[0, MAX_DEFAULT], 'io': [0, MAX_DEFAULT]}

def _get_opt(params, key, default):
    """Get optional parameter from dict"""
    if key in params:
        return params[key]
    return default

def _get_mand(params, key):
    """Get mandatory parameter from dict"""
    if key in params:
        return params[key] 
    raise KeyError('Parameter dict is missing entry for \'{}\''.format(key))
    
def _get_warns(limits, checks):
    """Check parameter values against limits"""
    warn = ''
    keys = list(checks.keys())
    for key in keys:
        lim = _get_opt(limits, key, [0, MAX_DEFAULT])
        if abs(checks[key]) > abs(lim[1]) or abs(checks[key]) < abs(lim[0]):
            warn += key + ' '
    return warn
        

class ElementMeta(type):
    """An element metaclass that will be used for element class creation.
    """
    def __instancecheck__(cls, instance):
        return cls.__subclasscheck__(type(instance))

    def __subclasscheck__(cls, subclass):
        return (hasattr(subclass, 'element_type') and
                hasattr(subclass, 'child_types') and 
                hasattr(subclass, 'from_file') and 
                callable(subclass.from_file)) 
                

class ElementInterface(metaclass=ElementMeta):
    """This interface is used for concrete element classes to inherit from.
    There is no need to define the ElementMeta methods of any class
    as they are implicitly made available via .__subclasscheck__().
    """
    pass


class Source:
    """The Source element must the root of a system. 
    A system can only have one source.
    
    Attributes
    ----------
    element_type : ElementTypes.SOURCE (enum)
        type of element
    """
    @property
    def element_type(self):
        """Defines the element type"""
        return ElementTypes.SOURCE
    
    @property
    def child_types(self):
        """Defines allowable child element types"""
        et = list(ElementTypes)
        et.remove(ElementTypes.SOURCE)
        return et
           
    def __init__(self, name:str, *, vo:float, rs:float=RS_DEFAULT, limits:dict=LIMITS_DEFAULT):
        """Set source name, voltage, internal resistance and max current"""
        self.params = {}
        self.params['name'] = name
        self.params['vo'] = vo
        self.params['rs'] = abs(rs)
        self.limits = limits
        
    @classmethod
    def from_file(cls, name:str, *, fname:str=""):
        """Configure source from configuration file"""
        with open(fname, 'r') as f:
            config = toml.load(f)
        
        v = _get_mand(config, 'vo')
        r = _get_opt(config, 'rs', RS_DEFAULT)
        lim = _get_opt(config, 'limits', LIMITS_DEFAULT)
        return cls(name, vo=v, rs=r, limits=lim)
     
    def _get_inp_current(self):
        return 0.0
    
    def _get_outp_voltage(self):
        return self.params['vo']
    
    def _solv_inp_curr(self, vi, vo, io):
        """Calculate element input current from vi, vo and io"""
        return io
        
    def _solv_outp_volt(self, vi, ii, io):
        """Calculate element output voltage from vi, ii and io"""
        return self.params['vo'] - self.params['rs'] * io
    
    def _solv_pwr_loss(self, vi, vo, ii, io):
        """Calculate power and loss in element"""
        pwr = abs(vo * io)
        loss = self.params['rs'] * io * io
        if pwr > 0.0:
            return pwr, loss, (pwr - loss) / pwr
        return pwr, loss, 100.0
    
    def _solv_get_warns(self, vi, vo, ii, io):
        """Check limits"""
        return _get_warns(self.limits, {'ii':ii})
        
        
class PLoad:
    """The Load element .
    
    Attributes
    ----------
    element_type : ElementTypes.LOAD (enum)
        type of element
    """
    @property
    def element_type(self):
        """Defines the element type"""
        return ElementTypes.LOAD
    
    @property
    def child_types(self):
        """The Load element cannot have childs"""
        return [None]
    
    def __init__(self, name:str, *, pwr:float, limits:dict=LIMITS_DEFAULT):
        """Set load power"""
        self.params = {}
        self.params['name'] = name
        self.params['pwr'] = abs(pwr)
        self.limits = limits
        
    @classmethod
    def from_file(cls, name:str, *, fname:str=""):
        """Configure load from configuration file"""
        with open(fname, 'r') as f:
            config = toml.load(f)
        
        p = _get_mand(config, 'pwr') 
        lim = _get_opt(config, 'limits', LIMITS_DEFAULT)
        return cls(name, pwr=p, limits=lim)
        
    def _get_inp_current(self):
        return 0.0 
    
    def _get_outp_voltage(self):
        return 0.0
        
    def _solv_inp_curr(self, vi, vo, io):
        """Calculate element input current from vi, vo and io"""
        if vi != 0.0:
            return self.params['pwr'] / abs(vi)
        return 0.0
    
    def _solv_outp_volt(self, vi, ii, io):
        """Load output voltage is always 0"""
        return 0.0
    
    def _solv_pwr_loss(self, vi, vo, ii, io):
        """Calculate power and loss in element"""
        return abs(vi * ii), 0.0, 100.0
    
    def _solv_get_warns(self, vi, vo, ii, io):
        """Check limits"""
        return _get_warns(self.limits, {'vi':vi, 'ii':ii})
    
class ILoad(PLoad):
    """The Load element .
    
    Attributes
    ----------
    element_type : ElementTypes.LOAD (enum)
        type of element
    """
    def __init__(self, name:str, *, ii:float, limits:dict=LIMITS_DEFAULT):
        """Set load current"""
        self.params = {}
        self.params['name'] = name
        self.params['ii'] = abs(ii)
        self.limits = limits
        
    @classmethod
    def from_file(cls, name:str, *, fname:str=""):
        """Configure load from configuration file"""
        with open(fname, 'r') as f:
            config = toml.load(f)
        
        i = _get_mand(config, 'ii')
        lim = _get_opt(config, 'limits', LIMITS_DEFAULT)
        return cls(name, ii=i, limits=lim)
        
    def _get_inp_current(self):
        return self.params['ii']
    
    def _solv_inp_curr(self, vi, vo, io):
        return self.params['ii']
    
        
class Loss:
    """The Loss element 
    
    Attributes
    ----------
    element_type : ElementTypes.LOSS (enum)
        type of element
    
    """
    @property
    def element_type(self):
        """Defines the element type"""
        return ElementTypes.LOSS
    
    @property
    def child_types(self):
        """Defines allowable child element types"""
        et = list(ElementTypes)
        et.remove(ElementTypes.SOURCE)
        return et
    
    def __init__(self, name, *, rs:float=RS_DEFAULT, vdrop:float=VDROP_DEFAULT, limits:dict=LIMITS_DEFAULT):
        """Set series resistance and/or vdrop"""
        self.params = {}
        self.params['name'] = name
        self.params['rs'] = abs(rs)
        self.params['vdrop'] = abs(vdrop)
        self.limits = limits
        
    @classmethod
    def from_file(cls, name:str, *, fname:str=""):
        """Configure source from configuration file"""
        with open(fname, 'r') as f:
            config = toml.load(f)
        
        vd = _get_opt(config, 'vdrop', VDROP_DEFAULT)
        r = _get_opt(config, 'rs', RS_DEFAULT)  
        lim = _get_opt(config, 'limits', LIMITS_DEFAULT)
        return cls(name, rs=r, vdrop=vd, limits=lim)
        
    def _get_inp_current(self):
        return 0.0
        
    def _get_outp_voltage(self):
        return 0.0
        
    def _solv_inp_curr(self, vi, vo, io):
        """Calculate element input current from vi, vo and io"""
        return io # TODO: iq?
    
    def _solv_outp_volt(self, vi, ii, io):
        """Calculate element output voltage from vi, ii and io"""
        if vi >= 0.0:
            return vi - self.params['rs'] * io - self.params['vdrop']
        else:
            return vi + self.params['rs'] * io + self.params['vdrop']
    
    def _solv_pwr_loss(self, vi, vo, ii, io):
        """Calculate power and loss in element"""
        loss = abs(self.params['rs'] * ii * ii + self.params['vdrop'] * ii)
        pwr = abs(vi * ii)
        if pwr > 0.0:
            return 0.0, loss, (pwr - loss) / pwr
        return 0.0, loss, 0.0
    
    def _solv_get_warns(self, vi, vo, ii, io):
        """Check limits"""
        return _get_warns(self.limits, {'vi':vi, 'vo':vo, 'ii':ii, 'io':ii})
        
class Converter:  
    """The Converter element 
    
    Attributes
    ----------
    element_type : ElementTypes.LOSS (enum)
        type of element
    """
    @property
    def element_type(self):
        """Defines the element type"""
        return ElementTypes.CONVERTER
    
    @property
    def child_types(self):
        """Defines allowable child element types"""
        et = list(ElementTypes)
        et.remove(ElementTypes.SOURCE)
        return et
    
    def __init__(self, name:str, *, vo:float, eff:float, iq:float=IQ_DEFAULT, limits:dict=LIMITS_DEFAULT):
        """Set converter parameters"""
        self.params = {}
        self.params['name'] = name
        self.params['vo'] = vo
        if not (eff > 0.0):
            raise ValueError('Efficiency must be > 0.0')
        if not (eff < 1.0):
            raise ValueError('Efficiency must be < 1.0')
        self.params['eff'] = eff
        self.params['iq'] = abs(iq)
        self.limits = limits
        
    @classmethod
    def from_file(cls, name:str, *, fname:str=""):
        """Configure source from configuration file"""
        with open(fname, 'r') as f:
            config = toml.load(f)
        
        v = _get_mand(config, 'vo')
        e = _get_mand(config, 'eff')
        iq = _get_opt(config, 'iq', IQ_DEFAULT)
        lim = _get_opt(config, 'limits', LIMITS_DEFAULT)
        return cls(name, vo=v, eff=e, iq=iq, limits=lim)

    def _get_inp_current(self):
        return self.params['iq']
        
    def _get_outp_voltage(self):
        return self.params['vo']
           
    def _solv_inp_curr(self, vi, vo, io):
        """Calculate element input current from vi, vo and io"""
        ve = self.params['eff']*vi
        if ve > 0.0:
            return self.params['iq'] + abs(vo*io/ve)
        return 0.0
    
    def _solv_outp_volt(self, vi, ii, io):
        """Calculate element output voltage from vi, ii and io"""
        return self.params['vo']
    
    def _solv_pwr_loss(self, vi, vo, ii, io):
        """Calculate power and loss in element"""
        loss = abs(self.params['iq'] * vi + (ii - self.params['iq']) * vi * (1. - self.params['eff']))
        pwr = abs(vi * ii)
        if pwr > 0.0:
            return 0.0, loss, (pwr - loss) / pwr
        return 0.0, loss, 0.0
    
    def _solv_get_warns(self, vi, vo, ii, io):
        """Check limits"""        
        return _get_warns(self.limits, {'vi':vi, 'vo':vo, 'ii':ii, 'io':ii})
    
class LinReg:  
    """The Linear regulator element 
    
    Attributes
    ----------
    element_type : ElementTypes.LINREG (enum)
        type of element
    """
    @property
    def element_type(self):
        """Defines the element type"""
        return ElementTypes.LINREG
    
    @property
    def child_types(self):
        """Defines allowable child element types"""
        et = list(ElementTypes)
        et.remove(ElementTypes.SOURCE)
        return et
    
    def __init__(self, name, *, vo:float, vdrop:float=VDROP_DEFAULT, iq:float=IQ_DEFAULT, limits:dict=LIMITS_DEFAULT):
        """Set linear regulator parameters"""
        self.params = {}
        self.params['name'] = name
        self.params['vo'] = vo
        if not (abs(vdrop) < abs(vo)):
            raise ValueError('Voltage drop must be < vo')
        self.params['vdrop'] = abs(vdrop)
        self.params['iq'] = abs(iq)
        self.limits = limits
        
    @classmethod
    def from_file(cls, name:str, *, fname:str=""):
        """Configure source from configuration file"""
        with open(fname, 'r') as f:
            config = toml.load(f)
        
        v = _get_mand(config, 'vo')
        vd = _get_opt(condfig, 'vdrop', VDROP_DEFAULT)
        iq = _get_opt(config, 'iq', IQ_DEFAULT)
        lim = _get_opt(config, 'limits', LIMITS_DEFAULT)
        return cls(name, vo=v, vdrop=vd, iq=iq, limits=lim)
        
        
    def _get_inp_current(self):
        return self.params['iq']
        
    def _get_outp_voltage(self):
        return self.params['vo']
    
    def _solv_inp_curr(self, vi, vo, io):
        """Calculate element input current from vi, vo and io"""
        return io + self.params['iq']
    
    def _solv_outp_volt(self, vi, ii, io):
        """Calculate element output voltage from vi, ii and io"""
        return self.params['vo']
    
    def _solv_pwr_loss(self, vi, vo, ii, io):
        """Calculate power and loss in element"""
        loss = abs((vi - vo) * io + vi * self.params['iq'])
        pwr = abs(vi * ii)
        if pwr >= 0.0:
            return 0.0, loss, (pwr - loss) / pwr
        return 0.0, loss, 0.0
    
    def _solv_get_warns(self, vi, vo, ii, io):
        """Check limits"""
        return _get_warns(self.limits, {'vi':vi, 'vo':vo, 'ii':ii, 'io':ii})

In [145]:
(issubclass(PLoad, ElementInterface), issubclass(ILoad, ElementInterface), issubclass(Source, ElementInterface), 
    issubclass(Loss, ElementInterface), issubclass(Converter, ElementInterface), issubclass(LinReg, ElementInterface))

(True, True, True, True, True, True)

In [146]:
s = Source('test', vo=12)
hasattr(s, 'element_type'), s.child_types, s.limits

(True,
 [<ElementTypes.LOAD: 2>,
  <ElementTypes.LOSS: 3>,
  <ElementTypes.CONVERTER: 4>,
  <ElementTypes.LINREG: 5>],
 {'vi': [0, 1000000.0],
  'vo': [0, 1000000.0],
  'ii': [0, 1000000.0],
  'io': [0, 1000000.0]})

In [150]:
import rustworkx as rx
import numpy as np
from rich.tree import Tree

#def df_style(val):
#    return "font-weight: bold"

class System:
    """System to be analyzed.
    """
    
    def __init__(self, name, source):
        self.g = None
        if type(name) != str:
            raise ValueError("Error: Name \"{}\" of system must be a string!".format(name))
        elif not isinstance(source, Source):
            raise ValueError("Error: First element of system must be a source!")
        else:
            self.g = rx.PyDAG(check_cycle=True, multigraph=False, attrs={})
            pidx = self.g.add_node(source)
            self.g.attrs[source.params['name']] = pidx
            
    def __get_index(self, name):
        """Get node index from element name"""
        #for n in [0] + list(rx.descendants(self.g, 0)):
        #    if self.g.get_node_data(n).name == name:
        #        return n
        if name in self.g.attrs:
            return self.g.attrs[name]

        return -1
    
    def __chk_parent(self, parent):
        """Check if parent exists"""
        if type(parent) != str:
            raise ValueError("Error: Parent name must be a string!")
        # check if parent exists
        if not parent in self.g.attrs.keys():
            raise ValueError("Error: Parent name \"{}\" not found!".format(parent))
            return False
        
        return True
    
    # check if element name is valid
    def __chk_name(self, name):
        """Check if element name is valid"""
        # check name type
        if type(name) != str:
            raise ValueError("Error: Element name must be a string!")
            return False
        # check if exists exists
        pidx = self.__get_index(name)
        if name in self.g.attrs.keys():
            raise ValueError("Error: Element name \"{}\" is already used!".format(name))
            return False
        
        return True
                  
    def __get_childs(self, rev=True):
        """Get dict of parent/childs"""
        if rev == True:
            childs = list(reversed(rx.bfs_successors(self.g, 0)))
        else:
            childs = list(rx.bfs_successors(self.g, 0))
        cdict = {}
        for c in childs:
            cs = []
            for l in c[1]:
                cs += [self.g.attrs[l.params['name']]]
            cdict[self.g.attrs[c[0].params['name']]] = cs
        return cdict
    
    def __get_nodes(self):
        """Get list of nodes in system"""
        nodes = []
        for n in self.g.nodes():
            nodes += [self.g.attrs[n.params['name']]]
        return sorted(nodes)
    
    def __get_parents(self):
        """Get list of parent of each child"""
        nodes = self.__get_nodes()
        ps = np.zeros(max(nodes)+1, dtype=np.int32)
        for n in nodes:
            if n != 0:
                pname = [i for i in self.g.attrs if self.g.attrs[i]==n]
                ps[n] = case1.g.attrs[self.g.predecessors(self.g.attrs[pname[0]])[0].params['name']]
        return list(ps)
    
    def __get_edges(self):
        """Get list of element connections (edges)"""
        return list(reversed(rx.dfs_edges(self.g, 0)))
    
    def __get_leaves(self):
        """Get list of leaf nodes"""
        nodes = self.__get_nodes()
        ls = np.zeros(max(nodes)+1, dtype=np.int32)
        for n in nodes:
            if self.g.out_degree(n) == 0:
                ls[n] = 1
        return list(ls)
        
    def __sys_vars(self):
        """Get system variable lists"""
        vn = max(self.__get_nodes())+1 # highest node index + 1
        v = list(np.zeros(vn)) # voltages
        i = list(np.zeros(vn)) # currents
        return v, i 

    def __make_rtree(self, adj, node):
        """Create Rich tree"""
        tree = Tree(node)
        for child in adj.get(node, []):
            tree.add(self.__make_rtree(adj, child))
        return tree
    
    def add_element(self, parent:str, *, element):
        """Add element to system"""
        # check that parent exists
        if not self.__chk_parent(parent):
            raise ValueError("Error: Parent name does not exist!")
            return
        
        # check that element name is unique
        if not self.__chk_name(element.params['name']):
            raise ValueError("Error: Element name already taken!")
            return
        
        pidx = self.__get_index(parent)
        
        # check that parent allows element type as child
        if not element.element_type in self.g[pidx].child_types:
            raise ValueError("Error: Parent does not allow child of type {}!".format(element.element_type.name))
            return
        
        cidx = self.g.add_child(pidx, element, None)
        self.g.attrs[element.params['name']] = cidx
        
    def change_element(self, *, name:str, element):
        """Replace element with a new one"""
        # if element name changes, check that it is unique
        if name != element.params['name']:
            if not self.__chk_name(element.params['name']):
                raise ValueError("Error: Element name already taken!")
                return
            
        eidx = self.__get_index(name)
        # check that parent allows element type as child
        if eidx != 0:
            parents = self.__get_parents()
            if not element.element_type in self.g[parents[eidx]].child_types:
                raise ValueError("Error: Parent does not allow child of type {}!".format(element.element_type.name))
                return

        self.g[eidx] = element
        # replace node name in graph dict
        del[self.g.attrs[name]]
        self.g.attrs[element.params['name']] = eidx
        
    def del_element(self, *, name:str, del_childs: bool=True):
        eidx = self.__get_index(name)
        if eidx == -1:
            raise ValueError("Error: Element name does not exist!")
        if eidx == 0:
            raise ValueError("Error: Cannot delete source node!")
        parents = self.__get_parents()
        leaves = self.__get_leaves()
        childs = self.__get_childs()
        # if not leaf, check if child type is allowed by parent type
        if leaves[eidx] == 0:
            #print(childs[eidx], parents[eidx])
            for c in childs[eidx]:
                if not self.g[c].element_type in self.g[parents[eidx]].child_types:
                    raise ValueError("Error: Parent and child of element are not compatible!")
        # delete childs first if selected
        if del_childs:
            for c in rx.descendants(self.g, eidx):
                print(c, eidx)
                del[self.g.attrs[self.g[c].params['name']]]     
                self.g.remove_node(c)
        # delete node
        self.g.remove_node(eidx)
        del[self.g.attrs[name]]
        # restore links between new parent and childs, unless deleted
        if not del_childs:                     
            if leaves[eidx] == 0:
                for c in childs[eidx]:
                    self.g.add_edge(parents[eidx], c, None)
               
    def tree(self, name=''):
        """Print tree structure, starting from node name"""
        if not name == '':
            if not name in self.g.attrs.keys():
                raise ValueError("Error: Element name is not valid!")
            root = name
        else:
            root = self.g[0].params['name']
            
        adj = rx.bfs_successors(self.g, self.g.attrs[root])
        ndict = {}
        for i in adj:
            c = []
            for j in i[1]:
                c += [j.params['name']]
            ndict[i[0].params['name']] = c
        return self.__make_rtree(ndict, root)
        
    def __sys_init(self):
        """Create vectors of init values for solver"""
        v, i = self.__sys_vars()
        for n in self.__get_nodes():
            v[n] = self.g[n]._get_outp_voltage()
            i[n] = self.g[n]._get_inp_current()
        return v, i
    
    def __fwd_prop(self, v:float, i:float):
        """Forward propagation of voltages"""
        vo, _ = self.__sys_vars()
        # update output voltages (per node)
        for n in self.__nodes:
            if self.__leaves[n] == 1: # leaf
                if n == 0: # root             
                    vo[n] = self.g[n]._solv_outp_volt(.0, .0, .0)
                    #print('leaf, root:', vo[n], n)
                else:
                    p = self.__parents[n]
                    vo[n] = self.g[n]._solv_outp_volt(v[p], i[p], .0)
                    #print('leaf: ', vo[n], n)
            else:
                # add currents into childs
                isum = 0
                for c in self.__childs_f[n]: 
                    isum += i[c]
                if n == 0: # root
                    vo[n] = self.g[n]._solv_outp_volt(.0, .0, isum)
                    #print('root:', vo[n], n)
                else:
                    p = self.__parents[n]
                    vo[n] = self.g[n]._solv_outp_volt(v[p], i[p], isum)
                    #print('element:' , vo[n], n)
        
        return vo

    def __back_prop(self, v:float, i:float):
        """Backward propagation of currents"""
        _, io = self.__sys_vars()
        # update input currents (per edge)
        for e in self.__edges:
            if self.__leaves[e[1]] == 1: # leaf
                io[e[1]] = self.g[e[1]]._solv_inp_curr(v[e[0]], .0, .0)
            else:
                c = self.__childs_b[e[1]]
                io[e[1]] = self.g[e[1]]._solv_inp_curr(v[e[0]], v[e[1]], i[c[0]])
        # add currents into childs from root
        for c in self.__childs_b[0]:
            io[0] += i[c]
        
        return io
    
    def solve(self, *, vtol=1e-3, itol=1e-6, maxiter=1000):
        """Analyze system"""
        # create lists of element realtionships for use by __sys_function()
        self.__nodes = self.__get_nodes()
        self.__edges = self.__get_edges()
        self.__childs_f = self.__get_childs(rev = False)
        self.__childs_b = self.__get_childs(rev = True)
        self.__parents = self.__get_parents()
        self.__leaves = self.__get_leaves()
        # initial condition
        v, i = self.__sys_init()
        # solve system function
        iters = 1
        while iters <= maxiter:
            vi = self.__fwd_prop(v, i)
            ii = self.__back_prop(vi, i)
            iters += 1
            if np.allclose(np.array(v), np.array(vi), rtol=vtol) and np.allclose(np.array(i), np.array(ii), rtol=itol):
                break;
            v, i = vi, ii
            
        # calculate results for each node
        res = {}
        names, parent, typ, pwr, loss, eff, warn, vsi, iso = [], [], [], [], [], [], [], [], []
        for n in self.__nodes: # [vi, vo, ii, io]
            names += [self.g[n].params['name']]
            vi = v[n]
            vo = v[n]
            ii = i[n]
            io = i[n]
            
            if n == 0: # root 
                #p, l, e = self.g[n].solv_pwr_loss(v[n], v[n], i[n], i[n])
                parent += ['']
                #print(n, v[n], v[n], i[n], i[n], p, l)
                vi = v[n] + self.g[n].params['rs'] * ii
            elif self.__leaves[n] == 1: # leaf
                #p, l, e = self.g[n].solv_pwr_loss(v[self.__parents[n]], v[n], i[n], 0.0)
                vi = v[self.__parents[n]]
                io = 0.0
                parent += [self.g[self.__parents[n]].params['name']]
                #print(n, v[n], 0.0, i[n], 0.0, p, l)
            else:
                io = 0.0
                for c in self.__childs_f[n]: 
                    io += i[c]
                #p, l, e = self.g[n].solv_pwr_loss(v[self.__parents[n]], v[n], i[n], isum)
                vi = v[self.__parents[n]]
                parent += [self.g[self.__parents[n]].params['name']]
                #print(n, v[self.__parents[n]], v[n], i[n], isum, p, l)
                
            
            p, l, e = self.g[n]._solv_pwr_loss(vi, vo, ii, io)    
            pwr += [p]
            loss += [l]
            eff += [e]
            typ += [self.g[n].element_type.name]
            warn += [self.g[n]._solv_get_warns(vi, vo, ii, io)]
            vsi += [vi]
            iso += [io]
            
        # remove unused node indices
        vso, isi = [], []
        for n in range(len(v)):
            if n in self.__nodes:
                vso += [v[n]]
                isi += [i[n]]
            
        # report
        res['Element'] = names
        res['Type'] = typ
        res['Parent'] = parent
        res['Vin (V)'] = vsi
        res['Vout (V)'] = vso
        res['Iin (A)'] = isi
        res['Iout (A)'] = iso
        res['Power (W)'] = pwr #[res['Power (W)'].sum() - res['Power (W)'][0]]
        res['Loss (W)'] = loss #[res['Loss (W)'].sum()]
        res['Efficiency (%)'] = eff
        res['Warnings'] = warn
        #print(res)
        df = pd.DataFrame(res)
        tpwr = abs(vsi[0] * i[0])
        tloss = df['Loss (W)'].sum()
        w = 'None'
        if sum(df['Warnings'] != '') > 0:
            w = 'Yes'
        df.loc[len(df)] = ['System total', '', '', vsi[0], 0.0, i[0], 0.0, tpwr, tloss, (tpwr - tloss)/tpwr, w]
        
        return df

![case1](case_1.svg)

In [151]:
case1 = System('Bluetooth sensor', Source('3V coin', vo=3, rs=13e-3))
case1.add_element(parent='3V coin', element=Loss('Resistor', rs=33.0))
case1.add_element('Resistor', element=Converter('1.8V buck', vo=-1.8, eff=0.87, iq=12e-6))
#case1.add_element('1.8V buck', element=Loss('Resistor2', rs=5.0))
case1.add_element('1.8V buck', element=PLoad('MCU', pwr=27e-3))
case1.add_element('3V coin', element=Converter('5V boost', vo=5, eff=0.91, iq=42e-6))
case1.add_element('5V boost', element=ILoad('Sensor', ii=15e-3))
case1.add_element('5V boost', element=LinReg('LDO 2.5V', vo=2.5, vdrop=0.27, iq=150e-6))
case1.add_element('LDO 2.5V', element=PLoad('ADC', pwr=15e-3))
#case1.g.attrs
case1.tree()

3V coin
├── 5V boost
│   ├── LDO 2.5V
│   │   └── ADC
│   └── Sensor
└── Resistor
    └── 1.8V buck
        └── MCU

In [152]:
df = case1.solve()
df

,Element,Type,Parent,Vin (V),Vout (V),Iin (A),Iout (A),Power (W),Loss (W),Efficiency (%),Warnings
0,3V coin,SOURCE,,3.000000,2.999698,0.023226,0.023226,0.069671,0.000007,0.999899,
1,Resistor,LOSS,3V coin,2.999698,2.606365,0.011919,0.011919,0.000000,0.004688,0.868876,
2,1.8V buck,CONVERTER,Resistor,2.606365,-1.800000,0.011919,0.015000,0.000000,0.004066,0.869124,
3,MCU,LOAD,1.8V buck,-1.800000,0.000000,0.015000,0.000000,0.027000,0.000000,100.000000,
4,5V boost,CONVERTER,3V coin,2.999698,5.000000,0.011307,0.021150,0.000000,0.003167,0.906620,
5,Sensor,LOAD,5V boost,5.000000,0.000000,0.015000,0.000000,0.075000,0.000000,100.000000,
6,LDO 2.5V,LINREG,5V boost,5.000000,2.500000,0.006150,0.006000,0.000000,0.015750,0.487805,
7,ADC,LOAD,LDO 2.5V,2.500000,0.000000,0.006000,0.000000,0.015000,0.000000,100.000000,
8,System total,,,3.000000,0.000000,0.023226,0.000000,0.069678,0.027678,0.602771,None


In [118]:
#case1.change_element(name='MCU', element=Load('Big MCU', pwr=130e-3))
#case1.change_element(name='MCU', element=Loss('Resistor1', res=133.0))
#case1.change_element(name='1.8V buck', element=Loss('Resistor2', res=7.8))
#case1.add_element('ADC', element='dummy')
case1.del_element(name='5V boost', del_childs=False)
case1.tree()

3V coin
├── Sensor
├── LDO 2.5V
│   └── ADC
└── Resistor
    └── 1.8V buck
        └── MCU

In [119]:
case1.solve()

,Element,Type,Parent,Vin (V),Vout (V),Iin (A),Iout (A),Power (W),Loss (W),Efficiency (%),Warnings
0,3V coin,SOURCE,,3.000000,2.999570,0.03307,0.03307,0.099195,0.000014,0.999857,
1,Resistor,LOSS,3V coin,2.999570,2.606214,0.01192,0.01192,0.000000,0.004689,0.868863,
2,1.8V buck,CONVERTER,Resistor,2.606214,-1.800000,0.01192,0.01500,0.000000,0.004066,0.869124,vo
3,MCU,LOAD,1.8V buck,-1.800000,0.000000,0.01500,0.00000,0.027000,0.000000,100.000000,vi
4,Sensor,LOAD,3V coin,2.999570,0.000000,0.01500,0.00000,0.044994,0.000000,100.000000,
5,LDO 2.5V,LINREG,3V coin,2.999570,2.500000,0.00615,0.00600,0.000000,0.003447,0.813125,
6,ADC,LOAD,LDO 2.5V,2.500000,0.000000,0.00600,0.00000,0.015000,0.000000,100.000000,
7,System total,,,3.000000,0.000000,0.03307,0.00000,0.099210,0.012216,0.876866,Yes
